# Demo of function *write_geotiff_from_xr* from *./swiss_utils/data_cube_utilities/sdc_utilities.py*

*****

__This script is the "official demo" of a function. Please if you want to modify it, work on your own copy.__

This is a modified version of the original `write_geotiff_from_xr` function, the following improvements were done by BC:

- fixed pixel shift bug
- original band name added to band numbers
- compression option added (`compr` argument)

Documentation for a given function can be accessed simply by adding ? at the end of the function in a cell. e.g. `write_geotiff_from_xr?` or by selecting the function and pressing `Shift-Tab`.

In this demo Jupyter script, the user can either use the in-script function (below) or import it from ./swiss_utils/data_cube_utilities/sdc_utilities.py.

In [ ]:
# Make sure the script is using the proper kernel
try:
    %run ../swiss_utils/assert_env.py
except:
    %run ./swiss_utils/assert_env.py

In [ ]:
# Import modules

# reload module before executing code
%load_ext autoreload
%autoreload 2

# define modules locations (you might have to adapt define_mod_locs.py)
%run ../swiss_utils/define_mod_locs.py

from datetime import datetime
from IPython.display import HTML

import datacube
dc = datacube.Datacube()

from utils.data_cube_utilities.dc_mosaic import create_hdmedians_multiple_band_mosaic

from swiss_utils.data_cube_utilities.sdc_utilities import load_multi_clean
from swiss_utils.data_cube_utilities.sdc_advutils import draw_map

# AND THE FUNCTION
from swiss_utils.data_cube_utilities.sdc_utilities import write_geotiff_from_xr

The next cell contains the dataset configuration information:
- product
- geographical extent
- time period
- bands

You can generate it in three ways:
1. manually from scratch,
2. by manually copy/pasting the final cell content of the [config_tool](config_tool.ipynb) notebook,
3. by loading the final cell content of the [config_tool](config_tool.ipynb) notebook using the magic `# %load config_cell.txt`.

In [ ]:
%load config_cell.txt

In [ ]:
draw_map([min_lat, max_lat], [min_lon, max_lon], draw = False)[0]

In [ ]:
# Create a mosaic
dataset_in, clean_mask = load_multi_clean(dc = dc, products = product, time = [start_date, end_date],
                                          lon = [min_lon, max_lon], lat = [min_lat, max_lat],
                                          measurements = measurements)
print(dataset_in)

In [ ]:
# Generate a mosaic

mosaic = create_hdmedians_multiple_band_mosaic(dataset_in, clean_mask, operation='medoid')
del clean_mask

print(mosaic)

In [ ]:
# Let's export as geotiff seting up the CRS as the mosaic function dropped it

write_geotiff_from_xr(tif_path = 'mosaic_demo.tif',
                      dataset = mosaic,
                      crs = dataset_in.crs,
                      compr = 'DEFLATE')

# add a direct link using IPython module
# You might have to use Shift-Right click to Save Link as...
display(HTML("""<a href="mosaic_demo.tif" target="_blank" >download geotiff file</a>"""))

In [ ]:
# Let's re-assign the CRS attribute to the mosaic and export it again without
# the quality band (pointless after mosaic processing)

mosaic.attrs['crs'] = dataset_in.crs
del dataset_in

write_geotiff_from_xr(tif_path = 'mosaic_demo.tif',
                      dataset = mosaic,
                      bands = [item for item in mosaic.data_vars if item != 'pixel_qa'],
                      compr = 'DEFLATE')

# add a direct link using IPython module
# You might have to use Shift-Right click to Save Link as...
display(HTML("""<a href="mosaic_demo.tif" target="_blank" >download geotiff file</a>"""))

In [ ]:
# You can also generate netcdf to check exported pixels overlay perfectly
# using the default to_netcdf function
mosaic.to_netcdf('mosaic_demo.nc')

# add a direct link using IPython module
# You might have to use Shift-Right click to Save Link as...
display(HTML("""<a href="mosaic_demo.nc" target="_blank" >download netcdf file</a>"""))